In [ ]:
!pip install ultralytics

### Imports

In [ ]:
import numpy as np
import matplotlib as plt
import torch 
import cv2, random, os
import torch.nn as nn
from torchvision import models, datasets, transforms
from torchvision.datasets import GTSRB
from torch.utils.data import random_split
import pandas as pd
from torch.utils.data import DataLoader
import torch.optim as optim


### Loading Data

In [ ]:
data_dir = "/kaggle/input/vehicledetection/VehiclesDetectionDataset"
train_dir = os.path.join(data_dir, "Train")
test_dir = os.path.join(data_dir, "Test/images")
valid_dir = os.path.join(data_dir, "Train")
yaml_dir = "/kaggle/input/vehicledetection/VehiclesDetectionDataset/dataset.yaml"

### Training YOLO on the vehicles dataset

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt")

train_results = model.train(
    data=yaml_dir,  
    epochs=10,  
    imgsz=640,  
    device="cpu", 
)

### Evaluate the model's performance on the validation set


In [ ]:
metrics = model.val()

In [ ]:
results = model.val(
    data=yaml_dir,
    split='test',     # evaluate on test split
    imgsz=640,
    conf=0.25
)

# Print metrics summary
print("Precision:", results.box.map50)
print("mAP@50:", results.box.map50)
print("mAP@50-95:", results.box.map)

### Perform object detection on an image

In [ ]:
import matplotlib.pyplot as plt
import cv2, random, os

# Path to test images
test_path = "/kaggle/input/vehicledetection/VehiclesDetectionDataset/test/images"

image_files = [f for f in os.listdir(test_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
sample_imgs = random.sample(image_files, 8)

plt.figure(figsize=(20, 10))

for i, f in enumerate(sample_imgs):
    img_path = os.path.join(test_path, f)
    results = model.predict(source=img_path, conf=0.25, verbose=False)
    pred_img = results[0].plot()
    pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)

    plt.subplot(2, 4, i + 1)
    plt.imshow(pred_img)
    plt.title(f)
    plt.axis("off")

plt.tight_layout()
plt.show()
